# TUTORIAL : PyTorch computation using multiple GPUs

Tutorial from https://github.com/ovh/ai-training-examples/blob/main/notebooks/getting-started/pytorch/

### Step 1: Import PyTorch library

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

### Step 2: Check that you have GPU(s) available on your notebook

In [2]:
# Function returns True if GPU is available (False otherwise)

if torch.cuda.is_available():

    for device_index in range(torch.cuda.device_count()):
        device = 'cuda:{}'.format(device_index)
        device_name = torch.cuda.get_device_name(device)
        print('{} ({})'.format(device, device_name))

else:
    print("No GPU detected. Code will run on CPU")

No GPU detected. Code will run on CPU


### Step 3: Declare the neural network to train

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 4096)
        self.fc2 = nn.Linear(4096, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

### Step 4: Declare train and test functions

In [4]:
def train(model, device, train_loader, test_loader, lr=1.0, gamma=0.7):
    print()
    print('Train {}'.format(device))
    optimizer = optim.Adadelta(model.parameters(), lr=lr)
    scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
    for epoch in range(1, epochs + 1):
        train_one_epoch(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
        scheduler.step()

def train_one_epoch(model, device, train_loader, optimizer, epoch):
    losses = []
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        if batch_idx % 50 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    return losses
                

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

### Step 5: Load MNIST dataset

In [5]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

import os
# create data directory if it does not exist
if not os.path.exists('data'):
    os.makedirs('data')

dataset1 = datasets.MNIST('./data', train=True, download=True, transform=transform)
dataset2 = datasets.MNIST('./data', train=False, transform=transform)

### Step 6: Train model

In [6]:
import timeit

# Input batch size for training
batch_size = 128
# Input batch size for testing
test_batch_size = 1000
# Number of epochs to train
epochs = 1

train_loader = torch.utils.data.DataLoader(dataset1, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(dataset2, batch_size=test_batch_size)

# Check if running on CPU or GPU and assign device:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
model = Net().to(device)
variables = {
    'model': model, 'device': device, 'train_loader': train_loader, 'test_loader': test_loader,
}


In [7]:
# Execute code
exe_time = timeit.timeit(f'train(model, device, train_loader, test_loader)', globals=variables, number=1, setup="from __main__ import train")

# Results
print('Training took: {:.2f}s'.format(exe_time))


Train cpu
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.309468
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.230775
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.112454
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.198683
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.145765
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.107455
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.168479
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.109777
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.140758
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.065405

Test set: Average loss: 0.0617, Accuracy: 9790/10000 (98%)

Training took: 526.98s
